In [ ]:
!pip install -U bitsandbytes accelerate


In [ ]:
!pip install -q peft transformers accelerate bitsandbytes


In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes sentencepiece huggingface_hub


In [ ]:
from huggingface_hub import login
login()  # Paste your HF token here

In [ ]:
from google.colab import files
uploaded = files.upload()

import json
from datasets import Dataset

# Load your JSONL dataset (limited to 500 for Colab safety)
data = [json.loads(line) for line in open("mcq_generator_dataset.jsonl")][:500]
dataset = Dataset.from_list(data).train_test_split(test_size=0.1)

Saving mcq_generator_dataset.jsonl to mcq_generator_dataset (1).jsonl


In [ ]:
from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Format prompt + response for instruction fine-tuning
def preprocess(example):
    prompt = f"""### Instruction:
Generate a multiple choice question with 4 options and one answer based on the paragraph below.

Paragraph: {example['instruction']}

### Response:
{example['response']}"""
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# ✅ Remove old columns after mapping
tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# Setup LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [ ]:
from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def preprocess(example):
    prompt = f"""### Instruction:
Generate a multiple choice question with 4 options and one answer based on the paragraph below.

Paragraph: {example['instruction']}

### Response:
{example['response']}"""
    inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
model.push_to_hub("Lingesh-S/mcq-mistral-lora")
tokenizer.push_to_hub("Lingesh-S/mcq-mistral-lora")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Lingesh-S/mcq-mistral-lora/commit/d26da8b89ecf30d5c215ca68cf592c3858c48de5', commit_message='Upload tokenizer', commit_description='', oid='d26da8b89ecf30d5c215ca68cf592c3858c48de5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Lingesh-S/mcq-mistral-lora', endpoint='https://huggingface.co', repo_type='model', repo_id='Lingesh-S/mcq-mistral-lora'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

base_model = "mistralai/Mistral-7B-Instruct-v0.1"
adapter_repo = "Lingesh-S/mcq-mistral-lora"
offload_folder = "./offload"  # ✅ Disk offloading directory

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

# Bitsandbytes config
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_enable_fp32_cpu_offload=True,
)

# Load base model with disk offloading
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder=offload_folder
)

model = PeftModel.from_pretrained(model, adapter_repo)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = """
# Instruction:
Generate a multiple choice question with 4 options and one correct answer based on the paragraph below.

Paragraph: The Lok Sabha is the House of the People in India. It is one of the two houses of Parliament.

# Response:"""

output = pipe(prompt, max_new_tokens=150, do_sample=True, temperature=0.5)
print(output[0]["generated_text"])


Device set to use cuda:0



### Instruction:
Generate a multiple choice question with 4 options and one correct answer based on the paragraph below.

Paragraph: The Lok Sabha is the House of the People in India. It is one of the two houses of Parliament.

### Response:

What is the name of the House of the People in India?

a) The Rajya Sabha
b) The Lok Sabha
c) The Supreme Court
d) The President's House

Correct answer: b) The Lok Sabha
